# Capstone Project Import Data and Configureation

Before populating the database create it in postgresql 

 Scripts: 
 CREATE DATABASE capstone;
 GRANT ALL PRIVILEGES ON DATABASE capstone TO username;

Not nessasary but can also create the table columns example:

CREATE TABLE weather_data (
    YEAR VARCHAR(255),
    STATION_ID VARCHAR(255),
    DATE DATE,
    ELEMENT VARCHAR(255),
    DATA_VALUE FLOAT,
);

In [ ]:
# Gathering climate change data from noaaglobal average land temp and ocean temp
# Move csv data files into postgresql 

import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Configuration
CSV_DIR = "C:/Users/Alex/Desktop/WeatherData/NOAAGlobalTemp"  # Directory containing the extracted CSV files      # Update csv directory name 
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def process_csv_to_db(csv_dir, engine):
    """Read CSV files and load them into the PostgreSQL database."""
    for csv_file in os.listdir(csv_dir):
        if csv_file.endswith(".csv"):
            file_path = os.path.join(csv_dir, csv_file)
            print(f"Processing {file_path}...")

            try:
                # Read CSV into a DataFrame
                df = pd.read_csv(file_path, usecols=[0, 1], header=None, names=[
                    "date", "anomaly"
                ])
                
                # Extract the file name to use as the land or Ocean distinction 
                LO = os.path.splitext(csv_file)[0]
                
                # Add the "land_or_ocean" column based on file name
                df.insert(0, "land_or_ocean", LO)
                
                # Write to the PostgreSQL database                                                           # Update table name 
                table_name = "raw_noaa_global_temps"
                df.to_sql(table_name, engine, if_exists="append", index=False)
                print(f"Inserted {len(df)} rows into the database.")
            
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

def main():
    # Ensure the CSV directory exists
    if not os.path.exists(CSV_DIR):
        print(f"CSV directory '{CSV_DIR}' does not exist.")
        return
    
    # Process and load CSV files into the PostgreSQL database
    process_csv_to_db(CSV_DIR, engine)
    print(f"All files have been processed and inserted into the database.")

if __name__ == "__main__":
    main()



Processing C:/Users/Alex/Desktop/WeatherData/NOAAGlobalTemp\Land.csv...
Inserted 900 rows into the database.
Processing C:/Users/Alex/Desktop/WeatherData/NOAAGlobalTemp\Ocean.csv...
Inserted 900 rows into the database.
All files have been processed and inserted into the database.


In [5]:
# Gathering climate change data from AGGI co2 
# Move csv data files into postgresql 

import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Configuration
CSV_DIR = "C:/Users/Alex/Desktop/WeatherData/AGGI"  # Directory containing the extracted CSV files      # Update csv directory name 
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def process_csv_to_db(csv_dir, engine):
    """Read CSV files and load them into the PostgreSQL database."""
    for csv_file in os.listdir(csv_dir):
        if csv_file.endswith(".csv"):
            file_path = os.path.join(csv_dir, csv_file)
            print(f"Processing {file_path}...")

            try:
                # Read CSV into a DataFrame
                df = pd.read_csv(file_path, usecols=[0, 1,2,3,4,5,6], header=None, names=[
                    "date", "decimal_date", "average", "deseasonalized", "ndays", "sdev", "unc"
                ])
                
                # Write to the PostgreSQL database                                                           # Update table name 
                table_name = "raw_aggi_co2_levels"
                df.to_sql(table_name, engine, if_exists="append", index=False)
                print(f"Inserted {len(df)} rows into the database.")
            
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

def main():
    # Ensure the CSV directory exists
    if not os.path.exists(CSV_DIR):
        print(f"CSV directory '{CSV_DIR}' does not exist.")
        return
    
    # Process and load CSV files into the PostgreSQL database
    process_csv_to_db(CSV_DIR, engine)
    print(f"All files have been processed and inserted into the database.")

if __name__ == "__main__":
    main()



Processing C:/Users/Alex/Desktop/WeatherData/AGGI\AGGI_co2_mm_mlo.csv...
Inserted 802 rows into the database.
All files have been processed and inserted into the database.


In [10]:
# Add table assigning each state a climate region 

from sqlalchemy import create_engine, Table, MetaData, Column, String, CHAR
from sqlalchemy.orm import sessionmaker

# Database configuration (as you provided)
DB_CONFIG = {
    "username": "agowen",  # Replace with your PostgreSQL username
    "password": "Capstone",  # Replace with your PostgreSQL password
    "host": "localhost",  # Replace with your PostgreSQL host
    "port": 5432,         # Replace with your PostgreSQL port (default: 5432)
    "database": "capstone"  # Replace with your PostgreSQL database name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)
metadata = MetaData()

# Define the 'state_region' table schema
state_region = Table(
    'state_region_mapping', metadata,
    Column('state_code', CHAR(2), primary_key=True),  # Two-letter state code
    Column('state_name', String(255)),
    Column('region_name', String(255))
)

# Create the 'state_region' table if it doesn't exist
metadata.create_all(engine)

# Data to insert (State abbreviation, state name, and region)
regions_and_states = [
    ('Northeast', [
        ('CT', 'Connecticut'),
        ('DE', 'Delaware'),
        ('ME', 'Maine'),
        ('MD', 'Maryland'),
        ('MA', 'Massachusetts'),
        ('NH', 'New Hampshire'),
        ('NJ', 'New Jersey'),
        ('NY', 'New York'),
        ('PA', 'Pennsylvania'),
        ('RI', 'Rhode Island'),
        ('VT', 'Vermont'),
    ]),
    ('Upper Midwest', [
        ('IA', 'Iowa'),
        ('MI', 'Michigan'),
        ('MN', 'Minnesota'),
        ('WI', 'Wisconsin'),
    ]),
    ('Ohio Valley', [
        ('IL', 'Illinois'),
        ('IN', 'Indiana'),
        ('KY', 'Kentucky'),
        ('MO', 'Missouri'),
        ('OH', 'Ohio'),
        ('TN', 'Tennessee'),
        ('WV', 'West Virginia'),
    ]),
    ('Southeast', [
        ('AL', 'Alabama'),
        ('FL', 'Florida'),
        ('GA', 'Georgia'),
        ('NC', 'North Carolina'),
        ('SC', 'South Carolina'),
        ('VA', 'Virginia'),
    ]),
    ('Northern Rockies and Plains', [
        ('MT', 'Montana'),
        ('NE', 'Nebraska'),
        ('ND', 'North Dakota'),
        ('SD', 'South Dakota'),
        ('WY', 'Wyoming'),
    ]),
    ('South', [
        ('AR', 'Arkansas'),
        ('KS', 'Kansas'),
        ('LA', 'Louisiana'),
        ('MS', 'Mississippi'),
        ('OK', 'Oklahoma'),
        ('TX', 'Texas'),
    ]),
    ('Southwest', [
        ('AZ', 'Arizona'),
        ('CO', 'Colorado'),
        ('NM', 'New Mexico'),
        ('UT', 'Utah'),
    ]),
    ('Northwest', [
        ('ID', 'Idaho'),
        ('OR', 'Oregon'),
        ('WA', 'Washington'),
    ]),
    ('West', [
        ('CA', 'California'),
        ('NV', 'Nevada'),
    ]),
    ('Alaska', [
        ('AK', 'Alaska'),
    ]),
    ('Hawaii', [
        ('HI', 'Hawaii'),
    ])
]

# Connect to the database
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the state_region table
for region, states_list in regions_and_states:
    for state_code, state_name in states_list:
        # Check if the state already exists to avoid duplicates
        existing_state = session.query(state_region).filter_by(state_code=state_code).first()
        if not existing_state:
            session.execute(state_region.insert().values(state_code=state_code, state_name=state_name, region_name=region))

# Commit the transaction
session.commit()

# Close the session
session.close()

print("Data successfully inserted into the 'state_region' table.")


Data successfully inserted into the 'state_region' table.


In [ ]:
# Access Storm_Event_data from the NOAA website, fails due to the file not being accessable 
# Manually downloaded csv from https://www.arcgis.com/apps/dashboards/2f0a9f25eea3410ca0443bdce936f8e5 

import os
import ftplib
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def get_ftp_links(webpage_url):
    response = requests.get(webpage_url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('ftp://')]
    
    return links

def download_ftp_file(ftp_url, download_path):
    parsed_url = urlparse(ftp_url)
    ftp_host = parsed_url.hostname
    ftp_path = parsed_url.path
    filename = os.path.basename(ftp_path)
    
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    
    local_file = os.path.join(download_path, filename)
    
    with ftplib.FTP(ftp_host) as ftp:
        ftp.login()
        with open(local_file, 'wb') as f:
            ftp.retrbinary(f'RETR {ftp_path}', f.write)
    
    print(f'Downloaded: {filename}')

def main(webpage_url, download_folder):
    ftp_links = get_ftp_links(webpage_url)
    
    if not ftp_links:
        print("No FTP links found on the page.")
        return
    
    for ftp_link in ftp_links:
        try:
            download_ftp_file(ftp_link, download_folder)
        except Exception as e:
            print(f'Failed to download {ftp_link}: {e}')

if __name__ == "__main__":
    webpage_url = "ftp://ftp.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles"
    download_folder = "C:/Users/Alex/Desktop/WeatherData/Storm_Event_data"
    main(webpage_url, download_folder)


In [11]:
# Move Storm Event Data csv into postgresql 
# last recorded date is 2021-09

import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Configuration
CSV_DIR = "C:/Users/Alex/Desktop/WeatherData/Storm_Event_data"  # Directory containing the extracted CSV files      # Update csv directory name 
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def process_csv_to_db(csv_dir, engine):
    """Read CSV files and load them into the PostgreSQL database."""
    for csv_file in os.listdir(csv_dir):
        if csv_file.endswith(".csv"):
            file_path = os.path.join(csv_dir, csv_file)
            print(f"Processing {file_path}...")

            try:
                # Read CSV with the first row as column names
                df = pd.read_csv(file_path)
                
                # Write to the PostgreSQL database
                table_name = "raw_storm_event_data"                                                         # Update table name
                df.to_sql(table_name, engine, if_exists="append", index=False)
                print(f"Inserted {len(df)} rows into the database.")
            
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

def main():
    # Ensure the CSV directory exists
    if not os.path.exists(CSV_DIR):
        print(f"CSV directory '{CSV_DIR}' does not exist.")
        return
    
    # Process and load CSV files into the PostgreSQL database
    process_csv_to_db(CSV_DIR, engine)
    print(f"All files have been processed and inserted into the database.")

if __name__ == "__main__":
    main()

# Inserted 1,662,829 rows into the database.

Processing C:/Users/Alex/Desktop/WeatherData/Storm_Event_data\NOAA_Storm_Events_Database_1950-2021_v2_9123639897772849412.csv...
Inserted 1662829 rows into the database.
All files have been processed and inserted into the database.


In [12]:
# Process Storm Event Data in postgresql so that the OBJECTID, state_codes, date, event_type are returned. 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_filtered_table():
    try:
        with engine.connect() as connection:
            # Query to filter and transform data from the `weather_data` table
            query = text("""
            SELECT 
                "OBJECTID" as event_id, 
                CASE 
                    WHEN "STATE" = 'ALABAMA' THEN 'AL'
                    WHEN "STATE" = 'ALASKA' THEN 'AK'
                    WHEN "STATE" = 'ARIZONA' THEN 'AZ'
                    WHEN "STATE" = 'ARKANSAS' THEN 'AR'
                    WHEN "STATE" = 'CALIFORNIA' THEN 'CA'
                    WHEN "STATE" = 'COLORADO' THEN 'CO'
                    WHEN "STATE" = 'CONNECTICUT' THEN 'CT'
                    WHEN "STATE" = 'DELAWARE' THEN 'DE'
                    WHEN "STATE" = 'FLORIDA' THEN 'FL'
                    WHEN "STATE" = 'GEORGIA' THEN 'GA'
                    WHEN "STATE" = 'HAWAII' THEN 'HI'
                    WHEN "STATE" = 'IDAHO' THEN 'ID'
                    WHEN "STATE" = 'ILLINOIS' THEN 'IL'
                    WHEN "STATE" = 'INDIANA' THEN 'IN'
                    WHEN "STATE" = 'IOWA' THEN 'IA'
                    WHEN "STATE" = 'KANSAS' THEN 'KS'
                    WHEN "STATE" = 'KENTUCKY' THEN 'KY'
                    WHEN "STATE" = 'LOUISIANA' THEN 'LA'
                    WHEN "STATE" = 'MAINE' THEN 'ME'
                    WHEN "STATE" = 'MARYLAND' THEN 'MD'
                    WHEN "STATE" = 'MASSACHUSETTS' THEN 'MA'
                    WHEN "STATE" = 'MICHIGAN' THEN 'MI'
                    WHEN "STATE" = 'MINNESOTA' THEN 'MN'
                    WHEN "STATE" = 'MISSISSIPPI' THEN 'MS'
                    WHEN "STATE" = 'MISSOURI' THEN 'MO'
                    WHEN "STATE" = 'MONTANA' THEN 'MT'
                    WHEN "STATE" = 'NEBRASKA' THEN 'NE'
                    WHEN "STATE" = 'NEVADA' THEN 'NV'
                    WHEN "STATE" = 'NEW HAMPSHIRE' THEN 'NH'
                    WHEN "STATE" = 'NEW JERSEY' THEN 'NJ'
                    WHEN "STATE" = 'NEW MEXICO' THEN 'NM'
                    WHEN "STATE" = 'NEW YORK' THEN 'NY'
                    WHEN "STATE" = 'NORTH CAROLINA' THEN 'NC'
                    WHEN "STATE" = 'NORTH DAKOTA' THEN 'ND'
                    WHEN "STATE" = 'OHIO' THEN 'OH'
                    WHEN "STATE" = 'OKLAHOMA' THEN 'OK'
                    WHEN "STATE" = 'OREGON' THEN 'OR'
                    WHEN "STATE" = 'PENNSYLVANIA' THEN 'PA'
                    WHEN "STATE" = 'RHODE ISLAND' THEN 'RI'
                    WHEN "STATE" = 'SOUTH CAROLINA' THEN 'SC'
                    WHEN "STATE" = 'SOUTH DAKOTA' THEN 'SD'
                    WHEN "STATE" = 'TENNESSEE' THEN 'TN'
                    WHEN "STATE" = 'TEXAS' THEN 'TX'
                    WHEN "STATE" = 'UTAH' THEN 'UT'
                    WHEN "STATE" = 'VERMONT' THEN 'VT'
                    WHEN "STATE" = 'VIRGINIA' THEN 'VA'
                    WHEN "STATE" = 'WASHINGTON' THEN 'WA'
                    WHEN "STATE" = 'WEST VIRGINIA' THEN 'WV'
                    WHEN "STATE" = 'WISCONSIN' THEN 'WI'
                    WHEN "STATE" = 'WYOMING' THEN 'WY'
                    ELSE "STATE"
                END AS state_code,
                CONCAT("YEAR", CASE 
                    WHEN "MONTH_NAME" = 'January' THEN '01'
                    WHEN "MONTH_NAME" = 'February' THEN '02'
                    WHEN "MONTH_NAME" = 'March' THEN '03'
                    WHEN "MONTH_NAME" = 'April' THEN '04'
                    WHEN "MONTH_NAME" = 'May' THEN '05'
                    WHEN "MONTH_NAME" = 'June' THEN '06'
                    WHEN "MONTH_NAME" = 'July' THEN '07'
                    WHEN "MONTH_NAME" = 'August' THEN '08'
                    WHEN "MONTH_NAME" = 'September' THEN '09'
                    WHEN "MONTH_NAME" = 'October' THEN '10'
                    WHEN "MONTH_NAME" = 'November' THEN '11'
                    WHEN "MONTH_NAME" = 'December' THEN '12'
                    ELSE "MONTH_NAME"
                END) AS date,
                "EVENT_TYPE" as event_type
            FROM raw_storm_event_data;
        """)

            # Execute query and fetch the transformed data into a DataFrame
            df = pd.read_sql(query, connection)

            # Create a new table to store the transformed data
            new_table_name = "processed_storm_event_data"                                                        # update new table name if needed 
            df.to_sql(new_table_name, engine, if_exists="replace", index=False)
            print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_filtered_table()

if __name__ == "__main__":
    main()

Table 'processed_storm_event_data' has been created successfully.


In [ ]:
# Process climate change data into one table containing both global temps and co2 levels 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_full_climate_change_data_table():
    try:
        with engine.connect() as connection:
            # Query to filter and transform data from the `weather_data` table
            query = text("""
            select land.date, global_land_temp, global_ocean_temp, co2_levels  from (
(SELECT date, anomaly as global_land_temp FROM raw_noaa_global_temps where land_or_ocean = 'Land') as ocean
join
(SELECT date, anomaly as global_ocean_temp FROM raw_noaa_global_temps where land_or_ocean = 'Ocean') as land
on land.date = ocean.date
left join 
(SELECT date, deseasonalized as co2_levels FROM raw_aggi_co2_levels) as co2
on land.date = co2.date
);
        """)

            # Execute query and fetch the transformed data into a DataFrame
            df = pd.read_sql(query, connection)

            # Create a new table to store the transformed data
            new_table_name = "climate_change_data_combined"                                                        # update new table name if needed 
            df.to_sql(new_table_name, engine, if_exists="replace", index=False)
            print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_full_climate_change_data_table()

if __name__ == "__main__":
    main()


Table 'climate_change_data_combined' has been created successfully.


In [ ]:
# Combine weather event table, state region mapping and climate change data into one final table
# RAW DATA USED FOR CLIMATE CHAGE FACTORS 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_full_event_and_climate_data_table():
    try:
        with engine.connect() as connection:
            # Query to filter and transform data from the `weather_data` table
            query = text("""
            select climate_data.date as date, global_land_temp, global_ocean_temp, co2_levels, climate_data.state_code, region_name, event_type  from 
(select * from climate_change_data_combined,state_region_mapping) climate_data  -- cross joins the two tables to assign every date to every state
left join
(select state_code, date, event_type from processed_storm_event_data) as weather_event_data -- left joins to weather event table 
on climate_data.date = cast(weather_event_data.date as bigint)
	and climate_data.state_code = weather_event_data.state_code;
        """)

            # Execute query and fetch the transformed data into a DataFrame
            df = pd.read_sql(query, connection)

            # Create a new table to store the transformed data
            new_table_name = "event_climate_data_complete_part1"                                                        # update new table name if needed 
            df.to_sql(new_table_name, engine, if_exists="replace", index=False)
            print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_full_event_and_climate_data_table()

if __name__ == "__main__":
    main()


Table 'event_climate_data_complete_part1' has been created successfully.


In [ ]:
# Combine weather event table, state region mapping and climate change data into one final table
# SMOOTHED CLIMATE CHANGE FACTORS SCRIPT 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with PostgreSQL username
    "password": "Capstone",  # Replace with PostgreSQL password
    "host": "localhost",          # Replace with PostgreSQL host
    "port": 5432,                 # Replace with PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_full_event_and_climate_data_table():
    try:
        with engine.connect() as connection:
            # Query to filter and transform data from the `weather_data` table
            query = text("""
            select climate_data.date as date, global_land_temp, global_ocean_temp, co2_levels, climate_data.state_code, region_name, event_type  from 
(select * from (SELECT 
    date,
    SUBSTRING(CAST(date AS VARCHAR), 1, 4) AS year,
    AVG(co2_levels) OVER (PARTITION BY SUBSTRING(CAST(date AS VARCHAR), 1, 4)) AS co2_levels,
    AVG(global_land_temp) OVER (PARTITION BY SUBSTRING(CAST(date AS VARCHAR), 1, 4)) AS global_land_temp,
    AVG(global_ocean_temp) OVER (PARTITION BY SUBSTRING(CAST(date AS VARCHAR), 1, 4)) AS global_ocean_temp
FROM 
    climate_change_data_combined),state_region_mapping) climate_data  -- cross joins the two tables to assign every date to every state
left join
(select state_code, date, event_type from processed_storm_event_data) as weather_event_data -- left joins to weather event table 
on climate_data.date = cast(weather_event_data.date as bigint)
	and climate_data.state_code = weather_event_data.state_code;
        """)

            # Execute query and fetch the transformed data into a DataFrame
            df = pd.read_sql(query, connection)

            # Create a new table to store the transformed data
            new_table_name = "event_climate_data_complete_part2"                                                        # update new table name if needed 
            df.to_sql(new_table_name, engine, if_exists="replace", index=False)
            print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_full_event_and_climate_data_table()

if __name__ == "__main__":
    main()


Table 'event_climate_data_complete_part2' has been created successfully.


In [20]:
# SCRIPT THAT CREATES THE BEST FIT CLIMATE CHANGE TABLES 

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with your PostgreSQL username
    "password": "Capstone",  # Replace with your PostgreSQL password
    "host": "localhost",  # Replace with your PostgreSQL host
    "port": 5432,  # Replace with your PostgreSQL port (default: 5432)
    "database": "capstone",  # Replace with your PostgreSQL database name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def fetch_climate_change_table_as_df(table_name):
    """
    Retrieves a table from the PostgreSQL database and returns it as a Pandas DataFrame.
    Parameters:
    - table_name (str): Name of the table to fetch.
    Returns:
    - Pandas DataFrame containing the table data.
    """
    try:
        with engine.connect() as connection:
            query = text(f"SELECT * FROM {table_name} where date >= 196001 and date <= 202012;")
            df = pd.read_sql(query, connection)
            return df
    except Exception as e:
        print(f"Error: {e}")
        return None

def best_fit_line(df, date_column, y_column):
    """
    Applies linear regression to the given data and returns the predicted values.
    """
    df_fit = df.copy()

    # Handle null values in the y column
    df_fit = df_fit.dropna(subset=[date_column,y_column])

    # Convert date to YYMM format if needed
    #df_fit['numeric_date'] = pd.to_datetime(df_fit[date_column].astype(str), errors='coerce').dt.strftime('%y%m').astype(float)

    # Reshape the date column to 2D array for LinearRegression
    #X = df_fit[['numeric_date']].values.reshape(-1, 1)
    X = df_fit[[date_column]].values
    y = df_fit[y_column].values

    model = LinearRegression()
    model.fit(X, y)

    # Add predictions as a new column
    df_fit[f'{y_column}_predicted'] = model.predict(X)

    return df_fit

def create_predictions_table(df, date_column, columns_to_predict, table_name):
    """
    Creates a new table in the database with the predicted values for each column.
    """
    # Apply linear regression for each column
    for column in columns_to_predict:
        df = best_fit_line(df, date_column, column)
    
    # Only keep the relevant columns (date and the predicted columns)
    result_df = df[[date_column] + [f"{col}_predicted" for col in columns_to_predict]]

    # Write the result dataframe to a new table in the database
    try:
        with engine.connect() as connection:
            result_df.to_sql(table_name, connection, if_exists='replace', index=False)
            print(f"Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

def main():
    table_name = "climate_change_data_combined"  # Replace with actual table name
    new_table_name = "climate_change_data_best_fit_predictions"  # Replace with desired new table name
    df_climate_change_data = fetch_climate_change_table_as_df(table_name)
    
    if df_climate_change_data is not None:
        # Define the columns to predict
        columns_to_predict = ['global_land_temp', 'global_ocean_temp', 'co2_levels']
        
        # Create predictions and store them in the database
        create_predictions_table(df_climate_change_data, 'date', columns_to_predict, new_table_name)

if __name__ == "__main__":
    main()


Table 'climate_change_data_best_fit_predictions' created successfully.


In [22]:
# Combine weather event table, state region mapping and climate change data into one final table
# BEST FIT LINE FOR CLIMATE CHAGE FACTORS 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with your PostgreSQL username
    "password": "Capstone",  # Replace with your PostgreSQL password
    "host": "localhost",          # Replace with your PostgreSQL host
    "port": 5432,                 # Replace with your PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with your PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_full_event_and_climate_data_table():
    try:
        with engine.connect() as connection:
            # Query to filter and transform data from the `weather_data` table
            query = text("""
            select climate_data.date as date, global_land_temp_predicted as global_land_temp, global_ocean_temp_predicted as global_ocean_temp, co2_levels_predicted as co2_levels, climate_data.state_code, region_name, event_type  from 
(select * from climate_change_data_best_fit_predictions,state_region_mapping) climate_data  -- cross joins the two tables to assign every date to every state
left join
(select state_code, date, event_type from processed_storm_event_data) as weather_event_data -- left joins to weather event table 
on climate_data.date = cast(weather_event_data.date as bigint)
	and climate_data.state_code = weather_event_data.state_code;
        """)

            # Execute query and fetch the transformed data into a DataFrame
            df = pd.read_sql(query, connection)

            # Create a new table to store the transformed data
            new_table_name = "event_climate_data_complete_part3"                                                        # update new table name if needed 
            df.to_sql(new_table_name, engine, if_exists="replace", index=False)
            print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_full_event_and_climate_data_table()

if __name__ == "__main__":
    main()


Table 'event_climate_data_complete_part3' has been created successfully.


In [ ]:
# Create a testing table for the Model 

import pandas as pd
from sqlalchemy import create_engine, text

# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with your PostgreSQL username
    "password": "Capstone",  # Replace with your PostgreSQL password
    "host": "localhost",          # Replace with your PostgreSQL host
    "port": 5432,                 # Replace with your PostgreSQL port (default: 5432)
    "database": "capstone"      # Replace with your PostgreSQL database name                              # Update DB Name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def create_testing_table():
    try:
        with engine.connect() as connection:
            # Query to filter and transform data from the `weather_data` table
            query = text("""
                        select date, global_land_temp, global_ocean_temp, co2_levels, state_code ,region_name, 
                            case when event_type in ('Flood','Drought','Tornado','Blizzard',
                            'Ice Storm','Excessive Heat','Wildfire','Tropical Storm',
                            'Tropical Depression','Frost/Freeze','Flash Flood') then event_type
                            when event_type in ('Hurricane (Typhoon)','Hurricane') then 'Hurricane'
                            else null end as event_type
                        from (select * from event_climate_data_complete_part1 union all select * from event_climate_data_complete_part2 union all select * from event_climate_data_complete_part3 )
                            where date >= 199601
                            and date <= 202012
                            and state_code not in ('AK','HI'); 
                         """)

            # Execute query and fetch the transformed data into a DataFrame
            df = pd.read_sql(query, connection)

            # Create a new table to store the transformed data
            new_table_name = "_input_table"                                                        # update new table name if needed 
            df.to_sql(new_table_name, engine, if_exists="replace", index=False)
            print(f"Table '{new_table_name}' has been created successfully.")

    except Exception as e:
        print(f"Error: {e}")

def main():
    create_testing_table()

if __name__ == "__main__":
    main()


Table '_input_table_1_dataset' has been created successfully.


In [ ]:
# OUTPUTS A SQL QUERY TO CSV FILE 


import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os

def save_dataframe_to_excel(df, output_path):
    """
    Saves a pandas DataFrame to an Excel file with headers at the specified path.

    Parameters:
    df (pd.DataFrame): The DataFrame to save.
    output_path (str): Full path (including filename) for the output Excel file.
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the DataFrame as an Excel file
    df.to_excel(output_path, index=False, header=True)
    print(f"DataFrame saved to '{output_path}'")


# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with your PostgreSQL username
    "password": "Capstone",  # Replace with your PostgreSQL password
    "host": "localhost",  # Replace with your PostgreSQL host
    "port": 5432,  # Replace with your PostgreSQL port (default: 5432)
    "database": "capstone_db_testing",  # Replace with your PostgreSQL database name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def fetch_table_as_df():
    """
    Retrieves a table from the PostgreSQL database and returns it as a Pandas DataFrame
    Returns:
    - Pandas DataFrame containing the table data.
    """
    try:
        with engine.connect() as connection:
            #query = text(f"SELECT * FROM {table_name};")

            query = text(f"""
            select SUBSTRING(cast(date as varchar), 1, 4) AS year, 
        SUBSTRING(cast(date as varchar), 5, 2) AS month, 
		global_land_temp, 
        global_ocean_temp, 
        co2_levels, 
        state_code,  
        region_name,
		count(distinct(event_type)) as weather_event_count from _input_table_1_dataset
group by 1,2,3,4,5,6,7
        """)
            df = pd.read_sql(query, connection)
            return df
    except Exception as e:
        print(f"Error: {e}")
        return None

def main():
    df = fetch_table_as_df()
    
    if df is not None:
        print(df)

    # Save to Excel file
    save_dataframe_to_excel(df, 'C:/Users/Alex/Desktop/Home_Documents/_Tufts/DS-288/Data/Raw_Training_Data.xlsx')

if __name__ == "__main__":
    main()


       year month  global_land_temp  global_ocean_temp  co2_levels state_code  \
0      1996    01              0.30               0.27      361.98         AL   
1      1996    01              0.30               0.27      361.98         AR   
2      1996    01              0.30               0.27      361.98         AZ   
3      1996    01              0.30               0.27      361.98         CA   
4      1996    01              0.30               0.27      361.98         CO   
...     ...   ...               ...                ...         ...        ...   
14395  2020    12              1.44               0.55      414.91         VT   
14396  2020    12              1.44               0.55      414.91         WA   
14397  2020    12              1.44               0.55      414.91         WI   
14398  2020    12              1.44               0.55      414.91         WV   
14399  2020    12              1.44               0.55      414.91         WY   

                       regi

In [ ]:
# OUTPUTS STATE AND REGION MAPPING TO A CSV FILE 


import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os

def save_dataframe_to_excel(df, output_path):
    """
    Saves a pandas DataFrame to an Excel file with headers at the specified path.

    Parameters:
    df (pd.DataFrame): The DataFrame to save.
    output_path (str): Full path (including filename) for the output Excel file.
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the DataFrame as an Excel file
    df.to_excel(output_path, index=False, header=True)
    print(f"DataFrame saved to '{output_path}'")


# Configuration
DB_CONFIG = {
    "username": "agowen",  # Replace with your PostgreSQL username
    "password": "Capstone",  # Replace with your PostgreSQL password
    "host": "localhost",  # Replace with your PostgreSQL host
    "port": 5432,  # Replace with your PostgreSQL port (default: 5432)
    "database": "capstone_db_testing",  # Replace with your PostgreSQL database name
}

# Connection string for PostgreSQL
CONN_STRING = f"postgresql://{DB_CONFIG['username']}:{DB_CONFIG['password']}@" \
              f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# Database setup
engine = create_engine(CONN_STRING)

def fetch_table_as_df(table_name):
    """
    Retrieves a table from the PostgreSQL database and returns it as a Pandas DataFrame.

    Parameters:
    - table_name (str): Name of the table to fetch.

    Returns:
    - Pandas DataFrame containing the table data.
    """
    try:
        with engine.connect() as connection:
            query = text(f"SELECT * FROM {table_name};")

            df = pd.read_sql(query, connection)
            return df
    except Exception as e:
        print(f"Error: {e}")
        return None

def main():
    
    table_name = "state_region_mapping"  # Replace with your actual table name
    df = fetch_table_as_df(table_name)
    
    if df is not None:
        print(df)

    # Save to Excel file
    save_dataframe_to_excel(df, 'C:/Users/Alex/Desktop/Home_Documents/_Tufts/DS-288/Data/state_mapping.xlsx')

if __name__ == "__main__":
    main()


   state_code      state_name                  region_name
0          CT     Connecticut                    Northeast
1          DE        Delaware                    Northeast
2          ME           Maine                    Northeast
3          MD        Maryland                    Northeast
4          MA   Massachusetts                    Northeast
5          NH   New Hampshire                    Northeast
6          NJ      New Jersey                    Northeast
7          NY        New York                    Northeast
8          PA    Pennsylvania                    Northeast
9          RI    Rhode Island                    Northeast
10         VT         Vermont                    Northeast
11         IA            Iowa                Upper Midwest
12         MI        Michigan                Upper Midwest
13         MN       Minnesota                Upper Midwest
14         WI       Wisconsin                Upper Midwest
15         IL        Illinois                  Ohio Vall